# y-y' distribution

In [ ]:
import sys
import os
from os.path import join
from pprint import pprint
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
import pandas as pd
import h5py
from scipy import ndimage
import proplot as pplt
from datetime import datetime

sys.path.append('..')
from tools import image_processing as ip
from tools import utils
from tools import plotting as mplt
from tools import energyVS06 as energy

sys.path.append('/Users/46h/Research/btf/btf-scripts/')
import scan_patterns as sp

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['cmap.discrete'] = False

In [ ]:
datadir = '../Diagnostics/Data/Measurements/2022-07-01/'
filenames = os.listdir(datadir)
filenames

In [ ]:
filename = '220701151627-y-emittance2d.h5'
file = h5py.File(join(datadir, filename), 'r')
data = file['scandata']
pprint(data.dtype.fields)

In [ ]:
acts = ['y_PositionSync', 'yp_PositionSync']
cam = 'cam06'
signame = f'{cam}_Integral'
sdiag = [f'{cam}_Integral', f'{cam}_Saturation', 'bcm04']
signal = data[signame]

In [ ]:
# Errors and warnings from log
for i in range(file['log'].size):
    if not(file['/log'][i, 'level'] == 'INFO'.encode('utf')):
        timestr = datetime.fromtimestamp(file['/log'][0, 'timestamp']).strftime("%m/%d/%Y, %H:%M:%S")
        print(f"{timestr} {file['log'][i, 'message']}")

# Configuration data
for key in file['/config'].keys():
    print(f"{key}")
    print("--------------")
    for name in file['/config'][key].dtype.names:
        print(f"{name}: {file['config'][key][name]}")
    print()

In [ ]:
cam_settings = ip.camera_settings(cam)
nx = cam_settings.nx
ny = cam_settings.ny
print(f'ny, nx = ({ny}, {nx})')

In [ ]:
fig, ax = pplt.subplots(figsize=(8, 2))
for act in acts:
    ax.plot(data[:, act], marker='.', ms=1, label=act, lw=0)
ax.legend()
ax.format(xlabel='Step', ylabel='[mm]')

In [ ]:
fig, ax = pplt.subplots()
ax.scatter(data[:, acts[0]], data[:, acts[1]], marker='s', s=50, 
           c=signal, norm='log', 
           colorbar=True, colorbar_kw=dict(label=signame))
ax.format(xlabel=acts[0], ylabel=acts[1])

In [ ]:
for item in sdiag:
    print(f'Diagnostic: {item}')
    print('Max = {:.3f}'.format(np.max(data[:, item])))
    print('Min = {:.3f}'.format(np.min(data[:, item])))
    print('Mean = {:.3f}'.format(np.mean(data[:, item])))
    fig, ax = pplt.subplots(figsize=(7.0, 2.0))
    ax.plot(data[:, item], color='black', marker='.', ms=0, label=item)
    if 'saturation' in item.lower():
        ax.set_ylim(0, 1)
    ax.format(xlabel='Step', ylabel=item)
    plt.show()

#### Pick frames for background calculation

In [ ]:
bgidx = np.arange(1, 35)

fig, ax = pplt.subplots()
ax.plot(data[:, acts[0]], data[:, acts[1]], alpha=0.2, color='grey')
ax.scatter(data[:, acts[0]], data[:, acts[1]], marker='s', 
           c=signal, norm='log',
           colorbar=True, colorbar_kw=dict(label=f'log10({signame})'))

ax.plot(data[bgidx, acts[0]], data[bgidx, acts[1]], 'rs', label='Background')
ax.legend()
ax.format(xlabel=acts[0], ylabel=acts[1])

## Threshold 

In [ ]:
# thr = 0.043
thr = 0.109
w = signal.copy()
w[w < thr] = 0.0
print(f'signame = {signame}')
print('Est 4D dynamic range 10^%.3f'%np.log10(thr / max(w)))

In [ ]:
idx, = np.where(signal >= thr)

In [ ]:
fig, ax = pplt.subplots(figsize=(8.0, 2.0))
ax.semilogy(signal, color='lightgray')
ax.semilogy(idx, signal[idx], marker='.', s=1, color='black', lw=0, label='Above thresh')
ax.legend()
ax.format(xlabel='Step', ylabel=signame)

In [ ]:
fig, ax = pplt.subplots(figsize=(8.0, 2.0))
ax.semilogy(np.sort(signal), color='lightgray')
ax.semilogy(np.sort(w), marker='.', color='black', lw=0, s=1, label='Above thresh')
ax.legend()
ax.format(xlabel='Step', ylabel=signame,)

In [ ]:
fig, axes = pplt.subplots(ncols=2)
for mask, ax in zip([False, True], axes):
    c = np.ma.masked_less_equal(signal, thr) if mask else signal
    ax.scatter(
        data[:, acts[0]], data[:, acts[1]], marker='s', s=50,
        c=c, norm='log',
        colorbar=True
    )
axes[1].format(xlim=axes[0].get_xlim(), ylim=axes[0].get_ylim())
axes.format(xlabel=acts[0], ylabel=acts[1])

## Measure against new scan boundaries

Make a list of coordinates that have signal.

In [ ]:
pts = np.vstack([data[:, acts[0]], data[:, acts[1]]]).T
sig_pts = pts[idx, :]

Define "normalized" actuator coordinates.

In [ ]:
reprate = 5.0
navg = 0
ndim = 2
slit_dict = {
    'y1': {
        'pvname': 'ITSF_Diag:Slit_HZ04',
        'center': 13.0,
        'distance': 25.0,
        'steps': 32,
        'min': -50.0,
        'max': +50.0,
    },
    'y2': {
        'pvname': 'ITSF_Diag:Slit_HZ06',
        'center': 15.0,
        'distance': 7.0,
        'steps': 32,
        'min': -50.0, 
        'max': +50.0,
    },
}
keys = list(slit_dict)
M = np.identity(ndim)
M[keys.index('y1'), keys.index('y2')] = 0.0  
M[keys.index('y2'), keys.index('y1')] = 0.85

center = np.array([slit_dict[key]['center'] for key in keys])
distance = np.array([slit_dict[key]['distance'] for key in keys])

In [ ]:
# Run points generator
kws = dict(
    slit_dict=slit_dict, 
    M=M, 
    reprate=reprate,
    navg=navg, 
    boundary='ellipsoid',  # {None, 'ellipsoid'} 
    inflate=1.05,
    exclude_outside_box=True,
)
lgen = list(sp.gen(**kws))

# Reshape
if navg > 0:
    new_pts = np.zeros((len(lgen), ndim))
    for i in range(len(lgen)):
        new_pts[i, :] = lgen[i][0]
else:
    lgen = np.array(lgen)
    new_pts = np.zeros((2 * lgen.shape[0], ndim))
    for i in range(ndim):
        new_pts[:, i] = lgen[:, i, :2].ravel()
    
# Un-shear / center new (generated) points.
Minv = np.linalg.inv(M)
new_pts_n = utils.apply(Minv, new_pts - center)

# Un-shear / center signal points.
pts_n = np.zeros(pts.shape)
for i in range(pts.shape[0]):
    pts_n[i, :] = np.matmul(Minv, pts[i, :] - center)
sig_pts_n = pts_n[idx, :]

# Get actuator coordinates.
coords = np.vstack([data[:, acts[0]], data[:, acts[1]]]).T
coords_n = np.zeros(coords.shape)
for row in range(coords.shape[0]):
    coords_n[row, :] = np.matmul(Minv, coords[row, :] - center)
    
# Compute ellipse coordinates.
_dist = distance.copy()
if 'inflate' in kws:
    _dist *= kws['inflate']
phi = np.linspace(0.0, 2.0 * np.pi, 100)
xx_n = 0.5 * _dist[0] * np.cos(phi) 
yy_n = 0.5 * _dist[1] * np.sin(phi)
xx, yy = utils.apply(M, np.vstack([xx_n, yy_n]).T).T

In [ ]:
# Normalize points to unit cube.
pts_nn = pts_n / (0.5 * distance)
radii = np.sqrt(np.sum(np.square(pts_nn), axis=1))
max_radius = np.max(radii[idx])
print(f'max radius with signal = {max_radius}')

fig, ax = pplt.subplots(figsize=(4, 2))
bins = 50
ax.hist(radii, color='lightgray', bins=bins, label='all')
ax.hist(radii[idx], color='black', bins=bins, label='above thresh')
ax.axvline(max_radius, color='pink', label='max radius')
ax.legend(loc='top')
ax.format(xlabel='radius', ylabel='num. points')
plt.show()

In [ ]:
fig, axes = pplt.subplots(ncols=2, wspace=None, figwidth=None, share=False)
c = np.ma.masked_less_equal(signal, thr) if mask else signal
for ax, _pts, _new_pts in zip(axes, [sig_pts, sig_pts_n], [new_pts, new_pts_n]):
#     ax.scatter(pts[:, 0], pts[:, 1], marker='s', c=signal, norm='log')
    ax.scatter(
        _pts[:, 0], _pts[:, 1], 
        marker='o', 
        ms=8,
        color='pink4',
        label='signal',
    )
    ax.plot(_new_pts[:, 0], _new_pts[:, 1], marker='.', s=3, lw=1.0, color='gray', alpha=0.4,
            label='scan')
axes.format(xlabel=acts[0], ylabel=acts[1], toplabels=['Sheared', 'Un-sheared'])
for i, (ax, x, y) in enumerate(zip(axes, [xx, xx_n], [yy, yy_n])):
    if i == 1:
        _center = [0.0, 0.0]
    else:
        _center = center
    ax.plot(x + _center[0], y + _center[1], color='black', ls='-', lw=1.0)
axes[0].legend(ncols=1, loc='upper left')
for ax in axes:
    xmin, xmax = ax.get_xlim()
    if xmin > xmax:
        xmin, xmax = xmax, xmin
    delta = 1.0
    xmin -= delta
    xmax += delta
    ymin, ymax = ax.get_ylim()
    ymin -= delta
    ymax += delta
    ax.format(xlim=(xmin, xmax), ylim=(ymin, ymax))
# plt.savefig(f'xxpp_ellipse_scan_navg{navg}.png')
plt.show()

In [ ]:
y1 = data['y_PositionSync'].copy()
y2 = data['yp_PositionSync'].copy()
print('y1_avg', np.average(y1[idx], weights=signal[idx]))
print('y2_avg', np.average(y2[idx], weights=signal[idx]))

## Convert to phase space coordinates 

In [ ]:
n = 32
y1_gv = np.linspace(np.min(y1), np.max(y1), int(1.1 * n) + 1)
y2_gv = np.linspace(np.min(y2), np.max(y2), int(2 * n) + 1)
Y1, Y2 = np.meshgrid(y1_gv, y2_gv, indexing='ij')

points = (y1, y2)
values = signal
new_points = (Y1.ravel(), Y2.ravel())
f = interpolate.griddata(points, values, new_points, method='linear', fill_value=0.0)
f = f.reshape(len(y1_gv), len(y2_gv))

fig, ax = pplt.subplots()
mplt.plot_image(f, x=Y1, y=Y2, ax=ax, colorbar=True)
ax.format(xlabel='y1 [mm]', ylabel='y2 [mm]')
plt.show()

In [ ]:
a2mm = 1.009  # assume same as first dipole
rho = 0.3556
GL05 = 0.0
GL06 = 0.0
l1 = 0.0
l2 = 0.0
l3 = 0.774
L2 = 0.311  # slit2 to dipole face
l = 0.129  # dipole face to VS06 screen (assume same for first/last dipole-screen)
LL = l1 + l2 + l3 + L2  # distance from emittance plane to dipole entrance

ecalc = energy.EnergyCalculate(l1=l1, l2=l2, l3=l3, L2=L2, l=l, amp2meter=a2mm*1e3)
Mslit = ecalc.getM1()  # slit-slit
Mscreen = ecalc.getM()  # slit-screen

# Watch the negative signs!
y = y1
yp = 1e3 * ecalc.calculate_yp(y1 * 1e-3, y2 * 1e-3, Mslit)  
y = y - np.mean(y)
yp = yp - np.mean(yp)

In [ ]:
points = np.vstack([y, yp]).T
values = signal

n = 37
y_gv = np.linspace(np.min(y), np.max(y), n)
yp_gv = np.linspace(np.min(yp), np.max(yp), int(n * 1.75))
Y, YP = np.meshgrid(y_gv, yp_gv, indexing='ij')
new_points = np.vstack([Y.ravel(), YP.ravel()]).T

f = interpolate.griddata(points, values, new_points, method='linear', fill_value=0.0)
f = f.reshape(len(y_gv), len(yp_gv))

fig, axes = pplt.subplots(ncols=2)
axes[0].scatter(y, yp, c=signal, marker='s')
f = np.clip(f - 0.05 * f.max(), 0.0, None)
mplt.plot_image(f / np.max(f), x=y_gv, y=yp_gv, ax=axes[1], colorbar=True, norm='log')
axes.format(xlabel='y [mm]', ylabel='yp [mrad]')